In [42]:
import feedparser
import requests
import re
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

from tqdm import tqdm

# Etape 1 : Extraction des flux RSS

In [6]:
def extract_rss_entries():
    urls = [
        "https://www.cert.ssi.gouv.fr/avis/feed",
        "https://www.cert.ssi.gouv.fr/alerte/feed"
    ]
    entries = []
    for url in urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            entries.append({
                'id': entry.link.split("/")[-2],
                'title': entry.title,
                'link': entry.link,
                'type': "Alerte" if "alerte" in url else "Avis",
                'date': entry.published
            })
    return entries

In [7]:
rss_entries = extract_rss_entries()
print(f"Nombre d'entrées RSS : {len(rss_entries)}\n")
# print(rss_entries)

Nombre d'entrées RSS : 80



# Etape 2 : Extraction des CVE

In [8]:
def extract_cves_from_json(entry_link):
    try:
        json_url = entry_link + "/json/"
        response = requests.get(json_url)
        data = response.json() # fichier JSON complet de l'entrée RSS

        # Extraction des CVE reference dans la clé cves du dict data
        cve_refs = data.get("cves", [])

        # Attention, il s’agit d’une liste des dictionnaires avec name et url comme clés
        print("CVE références :", cve_refs)
        print("Type de références", type(cve_refs))
        print("Type des éléments de la liste", type(cve_refs[0]))

        # Extraction des CVE avec une regex
        cve_pattern = r"CVE-\d{4}-\d{4,7}"
        cve_list = list(set(re.findall(cve_pattern, str(data))))
        print("CVE trouvés :", cve_list)

        # Retourne la liste de CVE référence pour l'entrée RSS entrée en argument + le JSON complet de l'entrée RSS
        return cve_refs, data
    except:
        return [], {}


In [9]:
first_entry = rss_entries[0]
cve_refs, full_json = extract_cves_from_json(first_entry["link"])
"""print(cve_refs)
print()
print(full_json)"""

"""for entry in rss_entries:
    cve_refs, full_json = extract_cves_from_json(entry["link"])
    print(cve_refs)
    print(full_json)"""

CVE références : [{'name': 'CVE-2025-4128', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4128'}, {'name': 'CVE-2025-4573', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4573'}]
Type de références <class 'list'>
Type des éléments de la liste <class 'dict'>
CVE trouvés : ['CVE-2025-4573', 'CVE-2025-4128']


'for entry in rss_entries:\n    cve_refs, full_json = extract_cves_from_json(entry["link"])\n    print(cve_refs)\n    print(full_json)'

# Etape 3 : Enrichissement des CVE

In [47]:
def enrich_cve(cve_id):
    mitre_data = {}
    epss_score = None

    # API CVE de Mitre
    try:
        mitre_url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
        r = requests.get(mitre_url)
        mitre_data = r.json()

        # Extraire la description de la CVE
        description = mitre_data["containers"]["cna"]["descriptions"][0]["value"]

        # Extraire le score et sévérité CVSS
        cvss_score = None
        cvss_severity = ""
        metrics = mitre_data["containers"]["cna"].get("metrics", [])
        for metric in metrics:
            cvss = metric.get("cvssV3_1") or metric.get("cvssV3_0")
            if cvss:
                cvss_score = cvss.get("baseScore")
                base_severity = cvss.get("baseSeverity")
                break

        # Extraire le CWE (type de vulnérabilité)
        cwe = "Non disponible"
        cwe_desc="Non disponible"
        problemtype = mitre_data["containers"]["cna"].get("problemTypes", {})
        if problemtype and "descriptions" in problemtype[0]:
            cwe = problemtype[0]["descriptions"][0].get("cweId", "Non disponible")
            cwe_desc=problemtype[0]["descriptions"][0].get("description", "Non disponible")

        # Extraire les produits affectés
        affected = mitre_data["containers"]["cna"]["affected"]
        for product in affected:
            vendor = product["vendor"]
            product_name = product["product"]
            versions = [v["version"] for v in product["versions"] if v["status"] == "affected"]
            print(f"Éditeur : {vendor}, Produit : {product_name}, Versions : {','.join(versions)}")

        print(f"CVE : {cve_id}")
        print(f"Description : {description}")
        print(f"Score CVSS : {cvss_score}")
        print(f"Type CWE : {cwe}")
        print(f"CWE Description : {cwe_desc}")
    except Exception as e:
        print(f"Erreur lors de l'enrichissement de {cve_id} : {e}")
        pass

    # API EPSS de First
    try:
        epss_url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
        r = requests.get(epss_url)
        epss_json = r.json()

        # Extraire la liste data du JSON (contenant le score EPSS)
        epss_data = epss_json.get("data", [])
        print(epss_data)

        # Si la liste n'est pas vide, extraire le score EPSS
        if epss_data:
            epss_score = epss_data[0]["epss"]
            print(f"CVE : {cve_id}")
            print(f"Score EPSS : {epss_score}")

        # Sinon, signaler le manque de score EPSS
        else:
            print(f"Aucun score EPSS trouvé pour {cve_id}")
    except Exception as e:
        print(f"Erreur lors de l'enrichissement de {cve_id} : {e}")
        epss_score = None

    return mitre_data, epss_score

In [48]:
cve_id_test = "CVE-2024-22024"
mitre_data_test, epss_score_test = enrich_cve(cve_id_test)

Éditeur : Ivanti, Produit : ICS, Versions : 9.1R14.5,9.1R17.3,9.1R18.4,22.1R6.1,22.2R4.1,22.3R1.1,22.4R1.1,22.5R1.2,22.6R1.1,22.4R2.3,22.5R2.3,22.6R2.2
Éditeur : Ivant , Produit : ICS, Versions : 9.1R15.3
Éditeur : Ivanti, Produit : IPS, Versions : 9.1R18.4,9.1R17.3,22.5R1.2
CVE : CVE-2024-22024
Description : An XML external entity or XXE vulnerability in the SAML component of Ivanti Connect Secure (9.x, 22.x), Ivanti Policy Secure (9.x, 22.x) and ZTA gateways which allows an attacker to access certain restricted resources without authentication.
Score CVSS : 8.3
Type CWE : Non disponible
CWE Description : Non disponible
[{'cve': 'CVE-2024-22024', 'epss': '0.943200000', 'percentile': '0.999380000', 'date': '2025-06-17'}]
CVE : CVE-2024-22024
Score EPSS : 0.943200000


# Etape 4 : Consolidation des données

In [1]:
def process_entries(entries):
    all_rows = []
    for entry in entries:
        # Récupérer la liste de CVE référence pour chaque entrée RSS
        cve_refs, full_json = extract_cves_from_json(entry['link'])

        # Si la liste de CVE est vide, on passe à l'entrée suivante
        if not cve_refs:
            print(f"Pas de CVE référence pour l'entrée RSS {entry['title']}, {entry['link']}")
            continue

        # Dans le cas ou la liste de CVE existe, nous ajoutons ses informations dans un DataFrame
        for cve in cve_refs:
            # Extraction du nom de la CVE (identifiant unique)
            cve_id = cve['name']

            # Extraction des données de l'API Mitre et du score EPSS à partir de l'identifiant de la CVE
            mitre_data, epss_score = enrich_cve(cve_id)

In [ ]:
def main():
    entries = extract_rss_entries()
    df = process_entries(entries)
    df.to_csv("data/final_cve_data.csv", index=False)
    print("✅ Données exportées dans final_cve_data.csv")

main()

  0%|          | 0/80 [00:00<?, ?it/s]

CVE références : [{'name': 'CVE-2025-4128', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4128'}, {'name': 'CVE-2025-4573', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-4573'}]
Type de références <class 'list'>
Type des éléments de la liste <class 'dict'>
CVE trouvés : ['CVE-2025-4573', 'CVE-2025-4128']


  1%|▏         | 1/80 [00:00<01:17,  1.02it/s]

CVE références : [{'name': 'CVE-2025-43003', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43003'}, {'name': 'CVE-2025-43007', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43007'}, {'name': 'CVE-2025-23191', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-23191'}, {'name': 'CVE-2025-42999', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-42999'}, {'name': 'CVE-2025-43009', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43009'}, {'name': 'CVE-2025-43011', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43011'}, {'name': 'CVE-2025-43006', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43006'}, {'name': 'CVE-2025-0060', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-0060'}, {'name': 'CVE-2025-30012', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-30012'}, {'name': 'CVE-2025-43000', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43000'}, {'name': 'CVE-2025-43004', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-43004'}, {'name': 'CVE-2025-31324', 'url': 'htt

  2%|▎         | 2/80 [00:10<07:52,  6.06s/it]

...

CVE références : [{'name': 'CVE-2023-53062', 'url': 'https://www.cve.org/CVERecord?id=CVE-2023-53062'}, {'name': 'CVE-2024-57981', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-57981'}, {'name': 'CVE-2025-22003', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-22003'}, {'name': 'CVE-2022-49790', 'url': 'https://www.cve.org/CVERecord?id=CVE-2022-49790'}, {'name': 'CVE-2024-57994', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-57994'}, {'name': 'CVE-2024-50223', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-50223'}, {'name': 'CVE-2025-21975', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21975'}, {'name': 'CVE-2022-49928', 'url': 'https://www.cve.org/CVERecord?id=CVE-2022-49928'}, {'name': 'CVE-2025-21980', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21980'}, {'name': 'CVE-2025-21889', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21889'}, {'name': 'CVE-2025-21861', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21861'}, {'name': 'CVE-2025-21937', 'url': 'h

 41%|████▏     | 33/80 [06:44<53:42, 68.56s/it]

CVE références : [{'name': 'CVE-2025-37943', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-37943'}, {'name': 'CVE-2024-26921', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-26921'}, {'name': 'CVE-2024-26645', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-26645'}, {'name': 'CVE-2025-22055', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-22055'}, {'name': 'CVE-2024-27042', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-27042'}, {'name': 'CVE-2025-37785', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-37785'}, {'name': 'CVE-2025-21997', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21997'}, {'name': 'CVE-2022-49395', 'url': 'https://www.cve.org/CVERecord?id=CVE-2022-49395'}, {'name': 'CVE-2025-21926', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21926'}, {'name': 'CVE-2025-21920', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-21920'}, {'name': 'CVE-2024-35930', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-35930'}, {'name': 'CVE-2023-52606', 'url': 'h

 42%|████▎     | 34/80 [06:48<37:43, 49.20s/it]

CVE références : [{'name': 'CVE-2025-26460', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-26460'}, {'name': 'CVE-2025-36887', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-36887'}, {'name': 'CVE-2025-32338', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32338'}, {'name': 'CVE-2024-48883', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-48883'}, {'name': 'CVE-2025-32314', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32314'}, {'name': 'CVE-2025-32316', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32316'}, {'name': 'CVE-2025-26785', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-26785'}, {'name': 'CVE-2025-32340', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32340'}, {'name': 'CVE-2025-32320', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32320'}, {'name': 'CVE-2025-32339', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-32339'}, {'name': 'CVE-2025-26459', 'url': 'https://www.cve.org/CVERecord?id=CVE-2025-26459'}, {'name': 'CVE-2025-32315', 'url': 'h